In [10]:
import simpy
from simpy import Resource
from ortools.linear_solver import pywraplp
#from graphviz import Graphviz
from networkx import DiGraph
import itertools
#from graphviz import DiGraph, Graphviz

# Define Plant, Warehouse, and Customer locations
plants = ["P1", "P2"]
warehouses = ["W1", "W2"]
customers = ["C1", "C2", "C3"]

# Define product demands
demand = {
    ("C1", "pr1"): 100,
    ("C1", "pr2"): 50,
    ("C2", "pr1"): 75,
    ("C2", "pr2"): 125,
    ("C3", "pr1"): 25,
    ("C3", "pr2"): 150,
}

# Define production capacities (units per day)
production_capacity = {
    ("P1", "pr1"): 150,
    ("P1", "pr2"): 200,
    ("P2", "pr1"): 100,
    ("P2", "pr2"): 125,
}

# Define transportation costs (per unit per km)
transport_cost = {
    ("P1", "W1"): 0.5,
    ("P1", "W2"): 0.7,
    ("P2", "W1"): 0.6,
    ("P2", "W2"): 0.8,
    ("W1", "C1"): 0.4,
    ("W1", "C2"): 0.3,
    ("W1", "C3"): 0.2,
    ("W2", "C1"): 0.3,
    ("W2", "C2"): 0.5,
    ("W2", "C3"): 0.6,
}

# Define initial inventory at warehouses (units)
inventory = {
    ("W1", "pr1"): 50,
    ("W1", "pr2"): 25,
    ("W2", "pr1"): 30,
    ("W2", "pr2"): 40,
}

# Define graph for visualization
G = DiGraph()
for node in plants + warehouses + customers:
    G.add_node(node)

# SimPy simulation setup
env = simpy.Environment()

# OR-Tools solver
solver = pywraplp.Solver.CreateSolver('CBC')

# Resources for each product at each location
resources = {}
for location, product in itertools.product(plants | warehouses | customers, ("pr1", "pr2")):
    resources[(location, product)] = Resource(capacity=inventory.get((location, product), 0) + production_capacity.get((location, product), 0), env= env)

# Processes for production, transportation, and demand fulfillment
def production(plant, product, env, resources):
    while True:
        yield env.timeout(1)  # Production takes one day
        resources[(plant, product)].put(production_capacity[(plant, product)])

def transportation(origin, destination, product, amount, env, resources):
    yield resources[(origin, product)].get(amount)
    yield env.timeout(transport_cost[origin, destination] * amount)
    resources[(destination, product)].put(amount)

def demand_fulfillment(customer, product, env, resources):
    yield resources[(customer, product)].get(demand[(customer, product)])
    print(f"Demand for {product} at {customer} met!")
    env.exit()  # Stop the simulation when demand is met

# Set up processes for each product and location
for plant, product in itertools.product(plants, ("pr1", "pr2")):
    env.process(production(plant, product, env, resources))

for origin, destination, product, amount in itertools.product(plants | warehouses, customers, ("pr1", "pr2"), demand.values()):
    env.process(transportation(origin, destination, product, amount, env, resources))

for customer, product in itertools.product(customers, ("pr1", "pr2")):
    env.process(demand_fulfillment(customer, product, env, resources))

# OR-Tools optimization (optional)
# Minimize total transportation cost
objective = solver.Objective()
for origin, destination, product, amount in itertools.product(plants | warehouses, customers, ("pr1", "pr2"), demand.values()):
    cost_var = solver.IntVar(0, solver.infinity(), f"cost_{origin}_{destination}_{product}_{amount}")
    objective.SetCoefficient(cost_var, transport_cost[origin, destination] * amount)
    solver.Add(transportation(origin, destination, product, amount, env, resources) == cost_var)

# Set production constraints
for plant, product in itertools.product(plants, ("pr1", "pr2")):
    product_production_var = solver.IntVar(0, production_capacity[(plant, product)], f"production_{plant}_{product}")
    solver.Add(resources[(plant, product)].put(product_production_var) <= product_production_var)

# Set demand constraints
for customer, product in itertools.product(customers, ("pr1", "pr2")):
    product_demand_var = solver.IntVar(0, demand[(customer, product)], f"demand_{customer}_{product}")
    solver.Add(resources[(customer, product)].get(product_demand_var) == product_demand_var)

# Solve the OR-Tools model
solver.Solve()

# Update inventory and production based on the solution
for origin, destination, product, amount in itertools.product(plants | warehouses, customers, ("pr1", "pr2"), demand.values()):
    cost_var = solver.SolutionValue(solver.IntVar(f"cost_{origin}_{destination}_{product}_{amount}"))
    if cost_var > 0:
        # Update inventory at origin and destination
        resources[(origin, product)].put(amount)
        resources[(destination, product)].get(amount)

# Update production based on the solution
for plant, product in itertools.product(plants, ("pr1", "pr2")):
    product_production_var = solver.SolutionValue(solver.IntVar(f"production_{plant}_{product}"))
    resources[(plant, product)].put(product_production_var)

# Update graph visualization with inventory and movement
for location, product in itertools.product(plants | warehouses | customers, ("pr1", "pr2")):
    inventory_amount = resources[(location, product)].level
    G.nodes[location]["label"] += f"\n{product}:{inventory_amount}"

for origin, destination, product, amount in itertools.product(plants | warehouses, customers, ("pr1", "pr2"), demand.values()):
    if solver.SolutionValue(solver.IntVar(f"cost_{origin}_{destination}_{product}_{amount}")) > 0:
        G.edges[origin, destination]["label"] = f"{product}:{amount}"

# Show the visualization
Graphviz(G).render("supply_chain.png")

env.run()

print("Simulation complete! All demands met.")


TypeError: unsupported operand type(s) for |: 'list' and 'list'